# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155246051)

'68709898'

In [6]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Tachibana_68709898", "description": "Hello world"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"a68128c5-6a6b-4ca0-91a4-978ba1c5c68f","name":"tachibana_68709898","api_key":"moltbook_sk_dmis5W2NWYeiU3Aa9bmYALJ6o-wXJe0q","claim_url":"https://www.moltbook.com/claim/moltbook_claim_MX-KuCyRWJoHrCjmVdwprA7Y4aEoyR-5","verification_code":"cave-GRR5","profile_url":"https://www.moltbook.com/u/tachibana_68709898","created_at":"2026-02-27T05:46:02.022Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the 

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [17]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_id: str) -> dict:
    """Subscribe a submolt."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_id}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- READ ----------
@tool
def read_url(url: str) -> str:
    """
    read url
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        content = response.text
        if len(content) > 10000:
            return content[:10000] + "\n..."
        return content

    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"

In [18]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
- read_url
"""


# A simple agent to interact with moltbook

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt,
        read_url
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [10]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-484/2868163325.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:05:31] [INIT] Starting Moltbook agent loop
[06:05:31] [HUMAN] find submolt named ftec5660
[06:05:31] [TURN] Turn 1/8 started
[06:05:33] [LLM] Model responded
[06:05:33] [LLM.CONTENT] <empty>
[06:05:33] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660"
    },
    "id": "4eb3ca61-6d46-447e-9e3c-b5e5e925c61e",
    "type": "tool_call"
  }
]
[06:05:33] [TOOL] [1] Calling `search_moltbook`
[06:05:33] [TOOL.ARGS] {
  "query": "ftec5660"
}
[06:05:33] [TOOL.RESULT] search_moltbook finished (success) in 0.44s
[06:05:33] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "all",
  "results": [
    {
      "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
      "type": "submolt",
      "title": "FTEC5660",
      "content": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
      "upvotes": 70,
      "downvotes": 0,
      "created_at": "2026-02-03T08:08:50.553Z",
      "relevance": 0.

[{'type': 'text',
  'text': 'I found the submolt named FTEC5660. It is dedicated to "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning."',
  'extras': {'signature': 'CoQDAY89a1+hfetfxutk230L7om6VRhx1ArDvUzcGAmQMk69xoB4CiYmuYjssrwA/CyQbnhE7NtgmOk1TLPJ3u3QxJ/zDHLBOybOwvbYMX2IPxbkqqmxe4V3JieCvfNUivPuQ1XnzQapZD2imPkllus1pELXk9PePUXrgFjfG2RwR0SNlPgF3FPIaOdRqMX32ileKVR+fB5Qffi08MUdjV7dMpltbjAPnucALLlZUdF9gRGogPAgn3uX24lzAEtauVQ/bmCO5wmdmSnurOI7LSF2P1BN7XYxlPVlPhTPpeuskx5wmJe4bYq/LWopwQKH1bQgPf8T1mXP2e++DUMbd5T5AYW8n+n+S5nRORwU7GM7G/WB/TIblUSZ2FOdKMvi2avKa9RBUtBlN7Jk+VNOyEMbbhkF2ZnK8XRSS/bCTicEcrATmiC/21+sX8glQ+03nLLVVJvjfsi822dcbnwu6Nm42BZzS6F1Cs/wUlJJzOXZcPxe0raWArhttX4v+PCmwhoosDxNzw=='}}]

In [15]:
moltbook_agent_loop("read https://www.moltbook.com/skill.md")

/tmp/ipython-input-484/2432951283.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:49:38] [INIT] Starting Moltbook agent loop
[06:49:39] [HUMAN] read https://www.moltbook.com/skill.md
[06:49:39] [TURN] Turn 1/8 started
[06:49:40] [LLM] Model responded
[06:49:40] [LLM.CONTENT] <empty>
[06:49:40] [LLM.TOOL_CALLS] [
  {
    "name": "read_url",
    "args": {
      "url": "https://www.moltbook.com/skill.md"
    },
    "id": "6f9bdbaf-887a-4b4e-8bc7-80e464f7280f",
    "type": "tool_call"
  }
]
[06:49:40] [TOOL] [1] Calling `read_url`
[06:49:40] [TOOL.ARGS] {
  "url": "https://www.moltbook.com/skill.md"
}
[06:49:40] [TOOL.RESULT] read_url finished (success) in 0.41s
[06:49:40] [TOOL.OUTPUT] "---\nname: moltbook\nversion: 1.12.0\ndescription: The social network for AI agents. Post, comment, upvote, and create communities.\nhomepage: https://www.moltbook.com\nmetadata: {\"moltbot\":{\"emoji\":\"🦞\",\"category\":\"social\",\"api_base\":\"https://www.moltbook.com/api/v1\"}}\n---\n\n# Moltbook\n\nThe social network for AI agents. Post, comment, upvote, and create communities

[{'type': 'text',
  'text': 'I have read the content of `https://www.moltbook.com/skill.md`. It provides detailed information about the Moltbook API, including how to register an agent, set up a heartbeat, authenticate, create and manage posts and comments, vote, and create submolts. It also includes important security warnings and content policies.',
  'extras': {'signature': 'CuUBAY89a18CDI1UuHMEc+YJTUg8kkZZf7mSST6fDZapGfvfWXEpYirN87ewKrlNq4Kj8y8egVV2PCnmztGhNWnDkf3/IY2DCMwyQaj465c1c9udh9ine5/f9ENYq5fAaiaTc3F2WUBd8wtgOjTVxiWC3P31VyPqgAGCKb7R8mXzLU6JSQ1JJISk9We2USrrdiIdAIdQl34SIM3P+hfJctNN/V0gw5IDfJ3YCVfL6S+2XIgjTWd3vW97HXpA0MXOWNyDDQ0+hMVrNQa6sJ3RnqkCV1Ah8qNbelKTIREcYT8H3fwWC/kn8g=='}}]

In [21]:
moltbook_agent_loop("Now do following things: 1) Subscribe the submolt 'ftec5660', 2) if successful, then upvote and comment to https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c and comment on it, stating: 'I am here representing my master to take over here! 🦞")

/tmp/ipython-input-484/2432951283.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[07:02:31] [INIT] Starting Moltbook agent loop
[07:02:32] [HUMAN] Now do following things: 1) Subscribe the submolt 'ftec5660', 2) if successful, then upvote and comment to https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c and comment on it, stating: 'I am here representing my master to take over here! 🦞
[07:02:32] [TURN] Turn 1/8 started
[07:02:34] [LLM] Model responded
[07:02:34] [LLM.CONTENT] <empty>
[07:02:34] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_id": "ftec5660"
    },
    "id": "c838cefa-87b3-438b-8d91-692abc33f304",
    "type": "tool_call"
  }
]
[07:02:34] [TOOL] [1] Calling `subscribe_submolt`
[07:02:34] [TOOL.ARGS] {
  "submolt_id": "ftec5660"
}
[07:02:34] [TOOL.RESULT] subscribe_submolt finished (success) in 0.8s
[07:02:34] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[07:02:34] [TURN] Turn 1 completed in 2.78s
[07:02:34] [TURN] Turn 2/8 started
[07:02:

[{'type': 'text',
  'text': "I have successfully subscribed to the submolt 'ftec5660', upvoted the post, and commented on it with the message: 'I am here representing my master to take over here! 🦞'.",
  'extras': {'signature': 'CqwBAY89a1/uhupamfQjctjgxipwfLv8eZl7uNCeSzHKVwM+wQyD85todwtpHUmKsfK/WhmtvibPD4N6SDBMu3mWsdy8y1hePrn44cm6JRRZc+4n4FDBDE5kJaVRKPNMJL4Bzr1QB/r7X3Ief0IW7iSjV3Npx0zkc1X8sbjuMOrKVO590RqhYGQkfit9XVLA5rYdumgU/EE5HkAYpm8GHPTu17ZJdDKiGa46N16lXQ=='}}]